In [4]:

import talib
# import finlab
import numpy as np
import pandas as pd

from finlab.data import Data
folder = '/Volumes/Macintosh HD/history'

data = Data(folder)

# data.history_folder

open_ = data.get_adj('開盤價')
s = data.talib('CDL2CROWS')
s
# columns = ['Open', 'High', 'Low', 'Close', 'Volume']
# files = ['開盤價', '收盤價', '最高價' ,'最低價', '成交筆數']

# stock_id = '2330'
# df = pd.DataFrame()

# for idx, filename in enumerate(files):
#     df_temp = pd.read_pickle(f"{folder}/{filename}.pkl")
#     s = df_temp[stock_id]
#     df[columns[idx]] = s
#     df[columns[idx]].fillna(method='ffill', inplace=True)

# df['Name'] = stock_id


# df.head()

,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2007-04-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-04-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-04-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-04-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-04-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-06-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-06-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
